In [1]:
from pyspark.sql import SparkSession
import zipfile


# def main():
spark = SparkSession.builder.appName("Exercise7").enableHiveSupport().getOrCreate()
zip_path = "data/hard-drive-2022-01-01-failures.csv.zip"

with zipfile.ZipFile(zip_path, "r") as z:
    csv_filename = z.namelist()[0] 
    with z.open(csv_filename) as f:
        data = f.read().decode("utf-8")

temp_path = "/tmp/temp_hard_drive.csv"
with open(temp_path, "w", encoding="utf-8") as temp_file:
    temp_file.write(data)

df = spark.read.csv(temp_path, header=True, inferSchema=True)

df.show(5)
df.printSchema()

# if __name__ == "__main__":
#     main()

25/11/11 11:17:13 WARN Utils: Your hostname, developer resolves to a loopback address: 127.0.1.1; using 192.168.29.61 instead (on interface enp3s0)
25/11/11 11:17:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/11 11:17:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/11 11:17:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------------+-------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-----

## Question1

In [2]:
from pyspark.sql import functions as F
df = df.withColumn("source_file", F.lit("hard-drive-2022-01-01-failures.csv"))
df.show(5, truncate=False)

+----------+-------------+-------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-----

## Question2

In [3]:
from pyspark.sql import functions as F

df = df.withColumn(
    "file_date",
    F.to_date(F.regexp_extract(F.col("source_file"), r"(\d{4}-\d{2}-\d{2})", 1), "yyyy-MM-dd")
)

df.select("source_file", "file_date").show(5, truncate=False)

+----------------------------------+----------+
|source_file                       |file_date |
+----------------------------------+----------+
|hard-drive-2022-01-01-failures.csv|2022-01-01|
|hard-drive-2022-01-01-failures.csv|2022-01-01|
|hard-drive-2022-01-01-failures.csv|2022-01-01|
|hard-drive-2022-01-01-failures.csv|2022-01-01|
|hard-drive-2022-01-01-failures.csv|2022-01-01|
+----------------------------------+----------+
only showing top 5 rows



## Question3

In [4]:
df = df.withColumn(
    "brand",
    F.when(
        F.size(F.split(F.col("model"), " ")) > 1,
        F.split(F.col("model"), " ")[0]
    ).otherwise("unknown")
)
df.show(5)

+----------+-------------+-------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-----

## Question 4

In [5]:
from pyspark.sql import functions as F, Window

df = df.withColumn("capacity_bytes", F.col("capacity_bytes").cast("long"))

ranked_df = df.filter(F.col("brand").isNotNull() & F.col("capacity_bytes").isNotNull())

windowSpec = Window.partitionBy("brand").orderBy(F.desc("capacity_bytes"))

ranked_df = ranked_df.withColumn("storage_ranking", F.dense_rank().over(windowSpec))

ranked_df.show(10)


# windowSpec = Window.partitionBy("brand").orderBy(F.desc("capacity_bytes"))
# ranked_df = (
#     df.filter(F.col("brand").isNotNull())
#       .withColumn("Ranking", F.dense_rank().over(windowSpec))
# )
# ranked_df.write.csv("reports/rank_on_capacity", header=True, mode="overwrite")
# ranked_df.show(3)


+----------+----------------+-----------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+----------

## Question 5

In [ ]:
# df = df.withColumn("primary_key", F.hash(F.col("date"), F.col("serial_number")))
# df.show(5)

window_spec = Window.orderBy(F.monotonically_increasing_id())

df_with_id = df.withColumn("row_number", F.row_number().over(window_spec))
df_with_hash = df_with_id.withColumn("primary_key", F.hash(F.col("row_number")))

df_with_hash

+----------+-------------+-------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-----